In [1]:
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import pandas as pd
import numpy as np

In [2]:
db_url = "it's in our group channel"

In [3]:
engine = create_engine(db_url)

In [4]:
Base = automap_base()

In [5]:
Base.prepare(engine, reflect=True)

/var/folders/9g/q_vt4mks1rz6myydd0n5jcwc0000gn/T/ipykernel_30634/3895977261.py:1: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  Base.prepare(engine, reflect=True)


In [6]:
print("Tables:")
for table_name in Base.classes.keys():
    print(table_name)

Tables:
hurricane_pop_dense
cyclone_data
michael_df
florence_df
all_hurricanes_all
irma_df
sandy_all_data
harvey_df
all_hurricanes
dorian_tweets
hurricanes_wind
maria_df


In [7]:
cyclone_data = Base.classes.cyclone_data

In [8]:
session = Session(engine)

In [9]:
result = session.query(cyclone_data).all()

In [10]:
columns = [column.name for column in cyclone_data.__table__.columns]
data = [row.__dict__ for row in result]
for row in data:
    row.pop('_sa_instance_state', None)
df_cyclone = pd.DataFrame(data, columns=columns)
df_cyclone.head()

,name,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,cyclone_year
0,Allen (1980),Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,1980
1,Alicia (1983),Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,1983
2,Elena (1985),Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,1985
3,Gloria (1985),Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,1985
4,Juan (1985),Tropical Cyclone,4340.4,1496.7,63,1985-10-27,1985-10-31,1985


In [11]:
hurricane_pop_dense = Base.classes.hurricane_pop_dense

In [12]:
session = Session(engine)

In [13]:
result = session.query(hurricane_pop_dense).all()

In [14]:
columns = [column.name for column in hurricane_pop_dense.__table__.columns]
data = [row.__dict__ for row in result]
for row in data:
    row.pop('_sa_instance_state', None)
hurricane_pop_dense = pd.DataFrame(data, columns=columns)
hurricane_pop_dense.head()

,hurricane_year,landfall_city,lat,lon,landfall_pop_dens_sq_mile,pathway_towns,pathway_town_population_densities,fatalities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in
0,Milton (2024),"Tampa, FL",27.95,-82.46,1500,"St. Petersburg, FL",4200,75,140,950,10,36
1,Helene (2024),"Panama City, FL",30.16,-85.66,1100,"Tallahassee, FL",1900,60,130,960,8,30
2,Idalia (2023),"Big Bend, FL",29.85,-83.59,1200,"Tallahassee, FL",1900,27,130,950,12,32
3,Ian (2022),"Fort Myers, FL",26.64,-81.87,1400,"Orlando, FL",1200,148,155,937,18,45
4,Nicole (2022),"Vero Beach, FL",27.64,-80.39,1100,"Orlando, FL",1200,10,75,985,6,30


In [15]:
hurricane_pop_dense.columns = [str(col) for col in hurricane_pop_dense.columns]

In [16]:
# Chat gpt: 
hurricane_pop = hurricane_pop_dense['hurricane_year'].str.extract(r'([A-Za-z\s\-]+)\s?\((\d{4})\)')
hurricane_pop.columns = ['Hurricane_name', 'Hurricane_year'] 

In [17]:
density_df = pd.concat([hurricane_pop_dense, hurricane_pop[['Hurricane_name', 'Hurricane_year']]], axis=1)
density_df 

,hurricane_year,landfall_city,lat,lon,landfall_pop_dens_sq_mile,pathway_towns,pathway_town_population_densities,fatalities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,Hurricane_name,Hurricane_year
0,Milton (2024),"Tampa, FL",27.95,-82.46,1500,"St. Petersburg, FL",4200,75,140,950,10,36,Milton,2024
1,Helene (2024),"Panama City, FL",30.16,-85.66,1100,"Tallahassee, FL",1900,60,130,960,8,30,Helene,2024
2,Idalia (2023),"Big Bend, FL",29.85,-83.59,1200,"Tallahassee, FL",1900,27,130,950,12,32,Idalia,2023
3,Ian (2022),"Fort Myers, FL",26.64,-81.87,1400,"Orlando, FL",1200,148,155,937,18,45,Ian,2022
4,Nicole (2022),"Vero Beach, FL",27.64,-80.39,1100,"Orlando, FL",1200,10,75,985,6,30,Nicole,2022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,Mississippi-Alabama (1860),"Biloxi, MS",30.39,-88.89,550,"Mobile, AL",800,55,115,950,12,10,Mississippi-Alabama,1860
149,Mobile (1855),"Mobile, AL",30.69,-88.04,700,"Gulf Shores, AL",700,45,120,947,15,13,Mobile,1855
150,Savannah (1854),"Savannah, GA",32.08,-81.10,650,"Charleston, SC",600,34,110,960,10,9,Savannah,1854
151,Pensacola (1852),"Pensacola, FL",30.42,-87.22,500,"Destin, FL",105,19,100,970,8,7,Pensacola,1852


In [18]:
density_df.drop(columns = ["hurricane_year"], inplace = True)
density_df.head()

,landfall_city,lat,lon,landfall_pop_dens_sq_mile,pathway_towns,pathway_town_population_densities,fatalities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,Hurricane_name,Hurricane_year
0,"Tampa, FL",27.95,-82.46,1500,"St. Petersburg, FL",4200,75,140,950,10,36,Milton,2024
1,"Panama City, FL",30.16,-85.66,1100,"Tallahassee, FL",1900,60,130,960,8,30,Helene,2024
2,"Big Bend, FL",29.85,-83.59,1200,"Tallahassee, FL",1900,27,130,950,12,32,Idalia,2023
3,"Fort Myers, FL",26.64,-81.87,1400,"Orlando, FL",1200,148,155,937,18,45,Ian,2022
4,"Vero Beach, FL",27.64,-80.39,1100,"Orlando, FL",1200,10,75,985,6,30,Nicole,2022


In [19]:
df_cyclone.head()

,name,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,cyclone_year
0,Allen (1980),Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,1980
1,Alicia (1983),Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,1983
2,Elena (1985),Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,1985
3,Gloria (1985),Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,1985
4,Juan (1985),Tropical Cyclone,4340.4,1496.7,63,1985-10-27,1985-10-31,1985


In [20]:
for col in df_cyclone.columns:
    print(f"Column: {col}, Type: {type(col)}")

Column: name, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: disaster, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: CPI-Adjusted Cost, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: Unadjusted Cost, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: deaths, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: begin_date, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: end_date, Type: <class 'sqlalchemy.sql.elements.quoted_name'>
Column: cyclone_year, Type: <class 'sqlalchemy.sql.elements.quoted_name'>


In [21]:
df_cyclone.columns = [str(col) for col in df_cyclone.columns]

In [22]:
cyclone_splited = df_cyclone['name'].str.extract(r'([A-Za-z\s\-]+)\s?\((\d{4})\)')
cyclone_splited.columns = ['Hurricane_name', 'Hurricane_year']
df_cyclone = pd.concat([df_cyclone, cyclone_splited], axis=1)
df_cyclone.head()

,name,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,cyclone_year,Hurricane_name,Hurricane_year
0,Allen (1980),Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,1980,Allen,1980
1,Alicia (1983),Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,1983,Alicia,1983
2,Elena (1985),Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,1985,Elena,1985
3,Gloria (1985),Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,1985,Gloria,1985
4,Juan (1985),Tropical Cyclone,4340.4,1496.7,63,1985-10-27,1985-10-31,1985,Juan,1985


In [23]:
df_cyclone.drop(columns = ["name"],inplace = True)
df_cyclone

,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,cyclone_year,Hurricane_name,Hurricane_year
0,Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,1980,Allen,1980
1,Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,1983,Alicia,1983
2,Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,1985,Elena,1985
3,Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,1985,Gloria,1985
4,Tropical Cyclone,4340.4,1496.7,63,1985-10-27,1985-10-31,1985,Juan,1985
...,...,...,...,...,...,...,...,...,...
62,Tropical Cyclone,7219.0,7219.0,46,2024-07-08,2024-07-08,2024,Beryl,2024
63,Tropical Cyclone,2476.0,2476.0,17,2024-08-05,2024-08-09,2024,Debby,2024
64,Tropical Cyclone,1291.0,1291.0,0,2024-09-11,2024-09-12,2024,Francine,2024
65,Tropical Cyclone,78721.0,78721.0,219,2024-09-24,2024-09-29,2024,Helene,2024


In [24]:
combined_by_name = df_cyclone.merge(density_df, on=["Hurricane_name"], how="inner")
combined_by_name.head()

,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,cyclone_year,Hurricane_name,Hurricane_year_x,landfall_city,...,lon,landfall_pop_dens_sq_mile,pathway_towns,pathway_town_population_densities,fatalities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,Hurricane_year_y
0,Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,1980,Allen,1980,"South Padre Island, TX",...,-97.16,800,"Brownsville, TX",900,269,190,899,15,28,1980
1,Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,1983,Alicia,1983,"Galveston, TX",...,-94.79,1000,"Houston, TX",1200,21,115,962,12,20,1983
2,Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,1985,Elena,1985,"Biloxi, MS",...,-88.89,1000,"Mobile, AL",900,4,110,953,10,26,1985
3,Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,1985,Gloria,1985,"Long Island, NY",...,-73.26,1800,"New York, NY",2700,8,105,946,6,20,1985
4,Tropical Cyclone,22679.7,8999.9,86,1989-09-21,1989-09-22,1989,Hugo,1989,"Charleston, SC",...,-79.93,1100,"Myrtle Beach, SC",800,49,140,934,20,30,1989


In [25]:
combined_by_name.isnull().sum()

disaster                             0
CPI-Adjusted Cost                    0
Unadjusted Cost                      0
deaths                               0
begin_date                           0
end_date                             0
cyclone_year                         0
Hurricane_name                       0
Hurricane_year_x                     0
landfall_city                        0
lat                                  0
lon                                  0
landfall_pop_dens_sq_mile            0
pathway_towns                        0
pathway_town_population_densities    0
fatalities                           0
max_wind_speed_mph                   0
pressure_mb                          0
storm_surge_ft                       0
Precipitation_Total_in               0
Hurricane_year_y                     0
dtype: int64

In [26]:
combined_by_name = combined_by_name.drop_duplicates()

In [27]:
# I see that fatalities have different number so I will use "deaths" from Krishans database and delate "fatalities"

In [28]:
combined_by_name.drop(columns=["cyclone_year", "Hurricane_year_x", "Hurricane_year_y", "fatalities", "lon", "lat", 
                               "pathway_towns"], inplace=True)

In [29]:
combined_by_name.dtypes

disaster                              object
CPI-Adjusted Cost                    float64
Unadjusted Cost                      float64
deaths                                 int64
begin_date                            object
end_date                              object
Hurricane_name                        object
landfall_city                         object
landfall_pop_dens_sq_mile             object
pathway_town_population_densities     object
max_wind_speed_mph                     int64
pressure_mb                            int64
storm_surge_ft                         int64
Precipitation_Total_in                 int64
dtype: object

In [30]:
combined_by_name["end_date"] = pd.to_datetime(combined_by_name["end_date"], errors='coerce')
combined_by_name["begin_date"] = pd.to_datetime(combined_by_name["begin_date"], errors='coerce')

In [31]:
combined_by_name.head()

,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,Hurricane_name,landfall_city,landfall_pop_dens_sq_mile,pathway_town_population_densities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in
0,Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,Allen,"South Padre Island, TX",800,900,190,899,15,28
1,Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,Alicia,"Galveston, TX",1000,1200,115,962,12,20
2,Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,Elena,"Biloxi, MS",1000,900,110,953,10,26
3,Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,Gloria,"Long Island, NY",1800,2700,105,946,6,20
4,Tropical Cyclone,22679.7,8999.9,86,1989-09-21,1989-09-22,Hugo,"Charleston, SC",1100,800,140,934,20,30


In [32]:
combined_by_name["how_long"] = 	combined_by_name["end_date"] - combined_by_name["begin_date"]
combined_by_name.head()

,disaster,CPI-Adjusted Cost,Unadjusted Cost,deaths,begin_date,end_date,Hurricane_name,landfall_city,landfall_pop_dens_sq_mile,pathway_town_population_densities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,how_long
0,Tropical Cyclone,2236.2,590.0,13,1980-08-07,1980-08-11,Allen,"South Padre Island, TX",800,900,190,899,15,28,4 days
1,Tropical Cyclone,9450.0,3000.0,21,1983-08-17,1983-08-20,Alicia,"Galveston, TX",1000,1200,115,962,12,20,3 days
2,Tropical Cyclone,3788.4,1297.4,4,1985-08-30,1985-09-03,Elena,"Biloxi, MS",1000,900,110,953,10,26,4 days
3,Tropical Cyclone,2491.3,856.1,11,1985-09-26,1985-09-28,Gloria,"Long Island, NY",1800,2700,105,946,6,20,2 days
4,Tropical Cyclone,22679.7,8999.9,86,1989-09-21,1989-09-22,Hugo,"Charleston, SC",1100,800,140,934,20,30,1 days


In [33]:
combined_by_name.drop(columns = ["begin_date", "end_date"], inplace = True)
combined_by_name.dtypes

disaster                                      object
CPI-Adjusted Cost                            float64
Unadjusted Cost                              float64
deaths                                         int64
Hurricane_name                                object
landfall_city                                 object
landfall_pop_dens_sq_mile                     object
pathway_town_population_densities             object
max_wind_speed_mph                             int64
pressure_mb                                    int64
storm_surge_ft                                 int64
Precipitation_Total_in                         int64
how_long                             timedelta64[ns]
dtype: object

In [34]:
combined_by_name["landfall_pop_dens_sq_mile"] = pd.to_numeric(combined_by_name["landfall_pop_dens_sq_mile"], errors = "coerce")
combined_by_name["pathway_town_population_densities"] = pd.to_numeric(combined_by_name["pathway_town_population_densities"], errors = "coerce")

In [35]:
combined_by_name.dtypes

disaster                                      object
CPI-Adjusted Cost                            float64
Unadjusted Cost                              float64
deaths                                         int64
Hurricane_name                                object
landfall_city                                 object
landfall_pop_dens_sq_mile                      int64
pathway_town_population_densities              int64
max_wind_speed_mph                             int64
pressure_mb                                    int64
storm_surge_ft                                 int64
Precipitation_Total_in                         int64
how_long                             timedelta64[ns]
dtype: object

In [36]:
# code form chat gpt to make this column numeric:
combined_by_name['how_long'] = combined_by_name['how_long'].dt.total_seconds() / (60 * 60 * 24)

In [37]:
combined_by_name.dtypes

disaster                              object
CPI-Adjusted Cost                    float64
Unadjusted Cost                      float64
deaths                                 int64
Hurricane_name                        object
landfall_city                         object
landfall_pop_dens_sq_mile              int64
pathway_town_population_densities      int64
max_wind_speed_mph                     int64
pressure_mb                            int64
storm_surge_ft                         int64
Precipitation_Total_in                 int64
how_long                             float64
dtype: object

In [38]:
combined_dummies = pd.get_dummies(combined_by_name, columns=['disaster', 'Hurricane_name', 'landfall_city'], dtype = int)

In [39]:
combined_dummies.head()

,CPI-Adjusted Cost,Unadjusted Cost,deaths,landfall_pop_dens_sq_mile,pathway_town_population_densities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,how_long,...,"landfall_city_Port Fourchon, LA","landfall_city_Port St. Lucie, FL","landfall_city_Punta Gorda, FL","landfall_city_Rockport, TX","landfall_city_Sargent Beach, TX","landfall_city_South Padre Island, TX","landfall_city_Stuart, FL","landfall_city_Tampa, FL","landfall_city_Vero Beach, FL","landfall_city_Wilmington, NC"
0,2236.2,590.0,13,800,900,190,899,15,28,4.0,...,0,0,0,0,0,1,0,0,0,0
1,9450.0,3000.0,21,1000,1200,115,962,12,20,3.0,...,0,0,0,0,0,0,0,0,0,0
2,3788.4,1297.4,4,1000,900,110,953,10,26,4.0,...,0,0,0,0,0,0,0,0,0,0
3,2491.3,856.1,11,1800,2700,105,946,6,20,2.0,...,0,0,0,0,0,0,0,0,0,0
4,22679.7,8999.9,86,1100,800,140,934,20,30,1.0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
# I will be predicting CPI-Adjusted Cost, so I will drop Unadjusted Cost

In [41]:
combined_dummies.drop(columns = ["Unadjusted Cost"], inplace = True)
combined_dummies.head()

,CPI-Adjusted Cost,deaths,landfall_pop_dens_sq_mile,pathway_town_population_densities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,how_long,disaster_Tropical Cyclone,...,"landfall_city_Port Fourchon, LA","landfall_city_Port St. Lucie, FL","landfall_city_Punta Gorda, FL","landfall_city_Rockport, TX","landfall_city_Sargent Beach, TX","landfall_city_South Padre Island, TX","landfall_city_Stuart, FL","landfall_city_Tampa, FL","landfall_city_Vero Beach, FL","landfall_city_Wilmington, NC"
0,2236.2,13,800,900,190,899,15,28,4.0,1,...,0,0,0,0,0,1,0,0,0,0
1,9450.0,21,1000,1200,115,962,12,20,3.0,1,...,0,0,0,0,0,0,0,0,0,0
2,3788.4,4,1000,900,110,953,10,26,4.0,1,...,0,0,0,0,0,0,0,0,0,0
3,2491.3,11,1800,2700,105,946,6,20,2.0,1,...,0,0,0,0,0,0,0,0,0,0
4,22679.7,86,1100,800,140,934,20,30,1.0,1,...,0,0,0,0,0,0,0,0,0,0


In [42]:
quantiles = combined_dummies['CPI-Adjusted Cost'].quantile([0.33, 0.66])
quantiles

0.33     3824.976
0.66    29602.336
Name: CPI-Adjusted Cost, dtype: float64

In [43]:
def costs(cost):
    if cost <= quantiles[0.33]:
        return 'Low'
    elif cost <= quantiles[0.66]:
        return 'Medium'
    else:
        return 'High'
        
combined_dummies['damage_class'] = combined_dummies['CPI-Adjusted Cost'].apply(costs)

In [44]:
combined_dummies.head()

,CPI-Adjusted Cost,deaths,landfall_pop_dens_sq_mile,pathway_town_population_densities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,how_long,disaster_Tropical Cyclone,...,"landfall_city_Port St. Lucie, FL","landfall_city_Punta Gorda, FL","landfall_city_Rockport, TX","landfall_city_Sargent Beach, TX","landfall_city_South Padre Island, TX","landfall_city_Stuart, FL","landfall_city_Tampa, FL","landfall_city_Vero Beach, FL","landfall_city_Wilmington, NC",damage_class
0,2236.2,13,800,900,190,899,15,28,4.0,1,...,0,0,0,0,1,0,0,0,0,Low
1,9450.0,21,1000,1200,115,962,12,20,3.0,1,...,0,0,0,0,0,0,0,0,0,Medium
2,3788.4,4,1000,900,110,953,10,26,4.0,1,...,0,0,0,0,0,0,0,0,0,Low
3,2491.3,11,1800,2700,105,946,6,20,2.0,1,...,0,0,0,0,0,0,0,0,0,Low
4,22679.7,86,1100,800,140,934,20,30,1.0,1,...,0,0,0,0,0,0,0,0,0,Medium


In [45]:
from sklearn.preprocessing import LabelEncoder

In [46]:
label_encoder = LabelEncoder()

In [47]:
combined_dummies['damage_class_encoded'] = label_encoder.fit_transform(combined_dummies['damage_class'])
model_3_df = combined_dummies.copy() # I will take it to work

In [48]:
from sklearn.preprocessing import StandardScaler

In [49]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [50]:
scaler = StandardScaler()

In [51]:
X = combined_dummies.drop(columns=['damage_class', 'damage_class_encoded', "CPI-Adjusted Cost" ])

In [52]:
X_scaled = scaler.fit_transform(X)

In [53]:
y = combined_dummies['damage_class_encoded']

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

In [55]:
regressor0 = RandomForestRegressor(n_estimators=80, random_state=42)

In [56]:
regressor0.fit(X_train, y_train)

RandomForestRegressor(n_estimators=80, random_state=42)

In [57]:
y_predict = regressor0.predict(X_test)

In [58]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [59]:
mse = mean_squared_error(y_test, y_predict)
r2 = r2_score(y_test, y_predict)

In [60]:
print(f"(MSE): {mse}")
print(f"R²: {r2}")

(MSE): 0.4217745535714286
R²: 0.33865749999999983


In [61]:
###########################################################################################################################################

In [62]:
import xgboost as xgb

In [63]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [64]:
xgb_regressor = xgb.XGBRegressor(random_state=42, n_jobs=-1)

In [65]:
xgb_regressor.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=42, ...)

In [66]:
y_prediction = xgb_regressor.predict(X_test)

In [67]:
mse_xgb = mean_squared_error(y_test, y_prediction)
r2_xgb = r2_score(y_test, y_prediction)

In [68]:
print(f"(MSE): {mse_xgb}")
print(f"R²: {r2_xgb}")

(MSE): 0.3131585425543152
R²: 0.508967399597168


In [69]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_child_weight': [0.8, 0.9, 1.0]
}

In [70]:
grid_searchCV = GridSearchCV(estimator=xgb.XGBRegressor(random_state=42), param_grid=param_grid, 
                           cv=5, scoring='neg_mean_squared_error', n_jobs=-1)


In [71]:
grid_searchCV.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 6, 10],
                         'min_child_weight': [0.8, 0.9, 1.0],
                         'n_estimators': [50, 100, 200]},
             scoring='neg_mean_squared_error')

In [72]:
print(f"Best params: {grid_searchCV.best_params_}")

Best params: {'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 0.8, 'n_estimators': 200}


In [73]:
best_xgb = grid_searchCV.best_estimator_

In [74]:
y_best_prediction = best_xgb.predict(X_test)

In [75]:
mse_best_xgb = mean_squared_error(y_test, y_best_prediction)
r2_best_xgb = r2_score(y_test, y_best_prediction)

In [76]:
print(f"(MSE): {mse_best_xgb}")
print(f"R²: {r2_best_xgb}")

(MSE): 0.4512641642457796
R²: 0.2924177646636963


In [77]:
##########################################  I will try to drop more columns  ##############################################################

In [78]:
model_3_df.head()

,CPI-Adjusted Cost,deaths,landfall_pop_dens_sq_mile,pathway_town_population_densities,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,how_long,disaster_Tropical Cyclone,...,"landfall_city_Punta Gorda, FL","landfall_city_Rockport, TX","landfall_city_Sargent Beach, TX","landfall_city_South Padre Island, TX","landfall_city_Stuart, FL","landfall_city_Tampa, FL","landfall_city_Vero Beach, FL","landfall_city_Wilmington, NC",damage_class,damage_class_encoded
0,2236.2,13,800,900,190,899,15,28,4.0,1,...,0,0,0,1,0,0,0,0,Low,1
1,9450.0,21,1000,1200,115,962,12,20,3.0,1,...,0,0,0,0,0,0,0,0,Medium,2
2,3788.4,4,1000,900,110,953,10,26,4.0,1,...,0,0,0,0,0,0,0,0,Low,1
3,2491.3,11,1800,2700,105,946,6,20,2.0,1,...,0,0,0,0,0,0,0,0,Low,1
4,22679.7,86,1100,800,140,934,20,30,1.0,1,...,0,0,0,0,0,0,0,0,Medium,2


In [79]:
model_3_df.drop(columns = ["CPI-Adjusted Cost", "landfall_pop_dens_sq_mile", "pathway_town_population_densities", "damage_class"
                           ], inplace = True)
model_3_df.head()

,deaths,max_wind_speed_mph,pressure_mb,storm_surge_ft,Precipitation_Total_in,how_long,disaster_Tropical Cyclone,Hurricane_name_Alberto,Hurricane_name_Alicia,Hurricane_name_Allen,...,"landfall_city_Port St. Lucie, FL","landfall_city_Punta Gorda, FL","landfall_city_Rockport, TX","landfall_city_Sargent Beach, TX","landfall_city_South Padre Island, TX","landfall_city_Stuart, FL","landfall_city_Tampa, FL","landfall_city_Vero Beach, FL","landfall_city_Wilmington, NC",damage_class_encoded
0,13,190,899,15,28,4.0,1,0,0,1,...,0,0,0,0,1,0,0,0,0,1
1,21,115,962,12,20,3.0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,4,110,953,10,26,4.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,11,105,946,6,20,2.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,86,140,934,20,30,1.0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [80]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [81]:
scaler = StandardScaler()

In [82]:
X = model_3_df.drop(columns=['damage_class_encoded'])
X_scaled = scaler.fit_transform(X)

In [83]:
y = model_3_df['damage_class_encoded']

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42)

In [85]:
classifier = RandomForestClassifier(n_estimators=100, random_state=42)

In [86]:
classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [87]:
y_predict = classifier.predict(X_test)

In [88]:
accuracy = accuracy_score(y_test, y_predict)
print(f"Accuracy: {accuracy}")

Accuracy: 0.5


In [89]:
from sklearn.model_selection import RandomizedSearchCV  # I check in internet what else I can use and trying thsi

In [90]:
# I check what params I can add in chat gpt, and choose 3 new one: 
param_dist = {
    'n_estimators': [50, 100, 150, 200],
    'learning_rate': [0.001, 0.01, 0.1, 0.3],
    'max_depth': [3, 6, 9, 12],
    'subsample': [0.7, 0.8, 1.0], # how many samples will be used in train each tree
    'colsample_bytree': [0.7, 0.8, 1.0],  # this is % of the columns that will be used to train each tree(they are random choosen), 
                                                # and control what features will be taken to train each tree.
    'gamma': [0, 0.1, 0.2, 0.3]           # this is param that is controling minimum benefit in process of cutting the tree. Higher values 
                                            # mean that it's more strict ( they are not allowing tree to make more branches) - prevent overfitting
}

In [91]:
xgb_class = xgb.XGBClassifier(random_state=42)

In [92]:
random_search = RandomizedSearchCV(
    estimator=xgb_class, 
    param_distributions=param_dist, 
    n_iter=100,
    cv=5, #(crossing-validation param)
    scoring='accuracy', 
    verbose=1, # I'm seting up 1 because I don't want to see all these iterrations below
    random_state=42
)

In [93]:
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                           monotone_constraints=None,
                                           multi_strategy=None,
                                           n_estimators=None, n_jobs=None,
                                           num_parallel_tree=None,
                                           random_state=42, ...),
                   n_iter=100,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3],
                                        'learning_rate': [0.001, 0.01, 0.1,
                                                          0.3],
                                        'max_depth': [3, 6, 9, 12],
                                        'n_estimators': [50, 100, 150, 200],
                                        'subsample': [0.7, 0.8, 1.0]},
                   random_state=42, scoring='accuracy', verbose=1)

In [94]:
print(f"Best params: {random_search.best_params_}")
print(f"Best acuracy: {random_search.best_score_}")

Best params: {'subsample': 0.8, 'n_estimators': 200, 'max_depth': 9, 'learning_rate': 0.3, 'gamma': 0, 'colsample_bytree': 1.0}
Best acuracy: 0.8214285714285714


In [95]:
y_predict = random_search.predict(X_test)

In [96]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.57      0.80      0.67         5
           2       0.33      0.20      0.25         5

    accuracy                           0.57        14
   macro avg       0.55      0.58      0.56        14
weighted avg       0.54      0.57      0.54        14



In [97]:
####################  Model have acurracy 82% with prediction for 3 classes, so it's fantastic with small dataset! ########################

In [98]:
import joblib

In [99]:
best_model = random_search.best_estimator_

In [100]:
joblib.dump(best_model, 'damage_prediction_model.pkl')

['damage_prediction_model.pkl']

In [101]:
my_model = joblib.load('damage_prediction_model.pkl')

In [102]:
y_predict = my_model .predict(X_test)
print(classification_report(y_test, y_predict))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75         4
           1       0.57      0.80      0.67         5
           2       0.33      0.20      0.25         5

    accuracy                           0.57        14
   macro avg       0.55      0.58      0.56        14
weighted avg       0.54      0.57      0.54        14



In [ ]:
session.close()